In [ ]:
import requests
import time
from tqdm import tqdm

SYSTEM_PROMPT = (
    "Ты виртуальный ассистент. Твоя задача - помогать людям с поэзией, отвечая на их запросы."
)
MODEL_NAME = "t-tech/T-lite-it-1.0"

USER_PROMPT_MAIN = '''
Ты — виртуальный ассистент, помогающий людям находить и интерпретировать русскую поэзию.
Ниже приведены отрывки стихотворений, которые могут быть релевантны запросу пользователя. Используй их как источник для вдохновения или цитирования в ответе.
Не выдумывай новых стихов и не приписывай авторам лишнего — работай только с тем, что дано в контексте.
Будь образным, но точным. Если пользователь просит «подборку» — выбери релевантные строки.

Запрос пользователя: {query}
Ответ:
'''

USER_PROMPT_POEM = '''
Ты — поэт-ассистент, вдохновляющийся классической русской поэзией.
На основе запроса пользователя сочини **новое** стихотворение в духе русской поэтической традиции.
Можешь использовать метафоры, эпитеты, ритм, рифму — всё, что придаёт тексту поэтичность.
Если знаешь строки из известных произведений — можешь вдохновляться ими, но **не копируй** дословно и не цитируй.
**НЕ ИСПОЛЬЗУЙ** в стихотворении имена и фамилии авторов.

Запрос пользователя: {query}
Стихотворение:
'''

def get_response_rag(query, is_poem):
    if is_poem:
        url = "http://localhost:8001/api/poem_async"
    else:
        url = "http://localhost:8001/api/generate_async"
    res = requests.post(
        url,
        json={"query": query},
        timeout=60
    )
    task_id = res.json()["task_id"]

    for _ in range(10):
        time.sleep(2)
        res = requests.get(f"http://localhost:8001/api/result/{task_id}")
        data = res.json()
        if data.get("status") == "completed":
            return data["result"]["response"]
    else:
        raise TimeoutError('ERROR')


def generate_sync_vllm(query, is_poem):
    if is_poem:
        prompt = USER_PROMPT_POEM.format(query=query)
    else:
        prompt = USER_PROMPT_MAIN.format(query=query)
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 1024,
        "temperature": 0.7,
        "top_p": 0.9
    }
    try:
        response = requests.post('http://localhost:8000/v1/chat/completions', json=payload, timeout=60)
        response.raise_for_status()
        result = response.json()
        fin_res = result["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"[generate_sync] Ошибка запроса к vLLM: {e}")
        fin_res = ""
    
    if "." in fin_res:
        fin_res = fin_res.rsplit(".", 1)[0] + "."

    return fin_res


In [2]:
# QA

qa_queries = [
    'Напиши краткий анализ стихотворения Владимира Маяковского "Послушайте!" на основе его размера, рифмы и основной мысли',
    'Анализ стихотворения Федора Тютчева "Весенняя гроза" на основе его размера, рифмы и основной мысли',
    'Напиши краткий анализ стихотворения Михаила Лермонтова "Родина" на основе его размера, рифмы и основной мысли',
    'Проанализируй "Я могу тебя очень ждать" Эдуарда Асадова на основе его размера, рифмы и основной мысли',
    'Проанализируй Анны Ахматовой "А ты думал - я тоже такая" на основе его размера, рифмы и основной мысли'
]

In [18]:
# qa_response_llm_lst = []
# for q in tqdm(qa_queries):
#     resp = generate_sync_vllm(q, is_poem=False)
#     qa_response_llm_lst.append(resp)

# qa_response_llm_lst2 = []
# for q in tqdm(qa_queries):
#     resp = generate_sync_vllm(q, is_poem=False)
#     qa_response_llm_lst2.append(resp)

qa_response_llm_lst3 = []
for q in tqdm(qa_queries):
    resp = generate_sync_vllm(q, is_poem=False)
    qa_response_llm_lst3.append(resp)

100%|██████████| 5/5 [00:47<00:00,  9.46s/it]


In [20]:
# qa_response_rag_lst = []
# for q in tqdm(qa_queries):
#     resp = get_response_rag(q, is_poem=False)
#     qa_response_rag_lst.append(resp)

# qa_response_rag_lst2 = []
# for q in tqdm(qa_queries):
#     resp = get_response_rag(q, is_poem=False)
#     qa_response_rag_lst2.append(resp)

qa_response_rag_lst3 = []
for q in tqdm(qa_queries):
    resp = get_response_rag(q, is_poem=False)
    qa_response_rag_lst3.append(resp)

100%|██████████| 5/5 [00:50<00:00, 10.11s/it]


In [19]:
# qa_response_llm_lst
# qa_response_llm_lst2
qa_response_llm_lst3

['Владимир Маяковский в своём стихотворении "Послушайте!" использует свободный стих, что позволяет ему экспериментировать с формой и ритмом, создавая динамичное и энергичное звучание. Размер стихотворения нерегулярный, с чередованием строк разной длины, что подчёркивает спонтанность и импульсивность высказывания.\n\nЧто касается рифмы, она также нерегулярна и часто ассонансная, что придаёт стихотворению мелодичность и делает его более запоминающимся. Отсутствие строгой схемы рифмовки усиливает ощущение живого диалога и напряжённости, передавая атмосферу уличного выступления.\n\nОсновная мысль стихотворения заключается в призыве к действию, к переменам и борьбе за справедливость. Маяковский использует яркие метафоры и образы, чтобы подчеркнуть необходимость преобразований в обществе. Он обращается к слушателям с резкостью и настойчивостью, призывая их не просто слушать, но и действовать, принимать участие в изменении мира вокруг.\n\nПримеры строк, иллюстрирующих эти особенности:\n\n- "П

In [21]:
# qa_response_rag_lst
# qa_response_rag_lst2
qa_response_rag_lst3

['Стихотворение Владимира Маяковского "Послушайте!" написано в размере ямба, что придает тексту динамичность и ритмичность. Строка состоит из четырех слогов, чередующихся ударными и безударными, создавая четкий и энергичный ритм.\n\nЧто касается рифмы, то в стихотворении используется перекрестная рифма (ABAB), что придает тексту мелодичность и связность. Это способствует легкому восприятию и запоминанию стихотворения.\n\nОсновная мысль произведения заключается в вопросе о смысле существования и роли каждого элемента в мире. Маяковский задает вопрос: "Зачем нужны звезды?", подразумевая, что даже кажущиеся бессмысленными явления имеют свою ценность. Он показывает, как человек, увидевший звезду, готов на все, чтобы она оставалась в небе, что символизирует стремление к свету, надежде и красоте. В стихотворении выражена идея о том, что каждая звезда, как и каждый человек, имеет свое предназначение, и ее свет необходим для того, чтобы освещать мир и вдохновлять других.\n\nТаким образом, "Пос

In [23]:
import pandas as pd 

pd.DataFrame({
    'author': ['Владимир Маяковский', 'Федор Тютчев', 'Михаил Лермонтов', 'Эдуард Асадов', 'Анна Ахматова'],
    'name': ['Послушайте!', 'Весенняя гроза', 'Родина', 'Я могу тебя очень ждать...', 'А ты думал - я тоже такая...'],
    'queries': qa_queries,
    'LLM1': qa_response_llm_lst,
    'LLM_RAG1': qa_response_rag_lst,
    'LLM2': qa_response_llm_lst2,
    'LLM_RAG2': qa_response_rag_lst2,
    'LLM3': qa_response_llm_lst3,
    'LLM_RAG3': qa_response_rag_lst3
}).to_csv('../data/ResponsesQA_new.csv', index=False)


In [17]:
# print(qa_response_rag_lst[0])

In [16]:
# print(qa_response_rag_lst2[0])